<h1>MUFG Skillset Sample</h1>

**The following example aims to show ability of working with:**

* Database Queries
* Excel Spreadsheets
* Python

In [1]:
# Import required tools
import pandas as pd
pd.set_option('display.max_columns', None)
from pandas import datetime
from datetime import *
import seaborn as sns
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
import pyodbc
import adodbapi
%load_ext sql
%config SqlMagic.autocommit=True
import numpy as np

today = date.today()
print('Date of last use:\n', today)

Date of last use:
 2020-05-18


In [2]:
# MSSQL Management Studio Connection

conn = pyodbc.connect('Driver={SQL server};'
                      'server=DESKTOP-BC2TA2S;'
                      'Database=BikeStores;'
                      'uid=PyXXXXXXXX;'
                      'pwd=PyXXXXXXXX;'
                      'trusted_connection=yes')
cursor = conn.cursor()
print('Connected to MSSQL - SSMS')

Connected to MSSQL - SSMS


In [3]:
# View all tables and find shared common key in order to be able to join

dfk = pd.read_sql_query('select * '
                        'from Information_Schema.Columns '
                        'order by TABLE_SCHEMA, column_name asc ', conn
                       )

dfk

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,ORDINAL_POSITION,COLUMN_DEFAULT,IS_NULLABLE,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,CHARACTER_OCTET_LENGTH,NUMERIC_PRECISION,NUMERIC_PRECISION_RADIX,NUMERIC_SCALE,DATETIME_PRECISION,CHARACTER_SET_CATALOG,CHARACTER_SET_SCHEMA,CHARACTER_SET_NAME,COLLATION_CATALOG,COLLATION_SCHEMA,COLLATION_NAME,DOMAIN_CATALOG,DOMAIN_SCHEMA,DOMAIN_NAME
0,BikeStores,dbo,vtabl,brand_id,3,None,NO,int,NaN,NaN,10.0,10.0,0.0,NaN,None,None,None,None,None,None,None,None,None
1,BikeStores,dbo,sysdiagrams,definition,5,None,YES,varbinary,-1.0,-1.0,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None
2,BikeStores,dbo,sysdiagrams,diagram_id,3,None,NO,int,NaN,NaN,10.0,10.0,0.0,NaN,None,None,None,None,None,None,None,None,None
3,BikeStores,dbo,vtb,first_name,1,None,NO,varchar,255.0,255.0,NaN,NaN,NaN,NaN,None,None,iso_1,None,None,SQL_Latin1_General_CP1_CI_AS,None,None,None
4,BikeStores,dbo,vtb,last_name,2,None,NO,varchar,255.0,255.0,NaN,NaN,NaN,NaN,None,None,iso_1,None,None,SQL_Latin1_General_CP1_CI_AS,None,None,None
5,BikeStores,dbo,vtabl,list_price,4,None,NO,decimal,NaN,NaN,10.0,10.0,2.0,NaN,None,None,None,None,None,None,None,None,None
6,BikeStores,dbo,vtabl,model_year,2,None,NO,smallint,NaN,NaN,5.0,10.0,0.0,NaN,None,None,None,None,None,None,None,None,None
7,BikeStores,dbo,sysdiagrams,name,1,None,NO,nvarchar,128.0,256.0,NaN,NaN,NaN,NaN,None,None,UNICODE,None,None,SQL_Latin1_General_CP1_CI_AS,None,None,None
8,BikeStores,dbo,sysdiagrams,principal_id,2,None,NO,int,NaN,NaN,10.0,10.0,0.0,NaN,None,None,None,None,None,None,None,None,None
9,BikeStores,dbo,vtabl,product_name,1,None,NO,varchar,255.0,255.0,NaN,NaN,NaN,NaN,None,None,iso_1,None,None,SQL_Latin1_General_CP1_CI_AS,None,None,None


In [4]:
# Select table Product and check number of records

rt1 = pd.read_sql_query('select count(brand_id) as Total_P_Rows '
                       'from BikeStores.production.products ', conn
                       )

rt1.head()

,Total_P_Rows
0,321


In [5]:
# Select table Brands and check number of records
rt2 = pd.read_sql_query('select count(brand_id) as Total_B_Rows '
                       'from BikeStores.production.brands ', conn
                       )

rt2.head()

,Total_B_Rows
0,9


In [6]:
# View contents of table products

t1 = pd.read_sql_query('select * '
                        'from BikeStores.production.products '
                        'order by product_name asc',conn
                      )
t1.head()

,product_id,product_name,brand_id,category_id,model_year,list_price
0,257,Electra Amsterdam Fashion 3i Ladies' - 2017/2018,1,3,2018,899.99
1,81,Electra Amsterdam Fashion 7i Ladies' - 2017,1,3,2017,1099.99
2,70,Electra Amsterdam Original 3i - 2015/2017,1,3,2017,659.99
3,82,Electra Amsterdam Original 3i Ladies' - 2017,1,3,2017,659.99
4,258,Electra Amsterdam Royal 8i - 2017/2018,1,3,2018,1259.90


In [7]:
# View dataset size
t1.shape

(321, 6)

In [8]:
# View contents of table products

t2 = pd.read_sql_query('select * '
                        'from BikeStores.production.brands '
                        'order by brand_id asc',conn
                      )
t2.head()

,brand_id,brand_name
0,1,Electra
1,2,Haro
2,3,Heller
3,4,Pure Cycles
4,5,Ritchey


In [9]:
# View dataset size

t2.shape

(9, 2)

In [10]:
# Based on results, we'll perform a left join to
# to preserve largest table content

join = pd.read_sql_query('select * '
                         'from production.products as t1 '
                         'left join production.brands as t2 '
                         'on t1.brand_id = t2.brand_id '
                         'order by product_name ', conn)

join.head()

,product_id,product_name,brand_id,category_id,model_year,list_price,brand_id,brand_name
0,257,Electra Amsterdam Fashion 3i Ladies' - 2017/2018,1,3,2018,899.99,1,Electra
1,81,Electra Amsterdam Fashion 7i Ladies' - 2017,1,3,2017,1099.99,1,Electra
2,70,Electra Amsterdam Original 3i - 2015/2017,1,3,2017,659.99,1,Electra
3,82,Electra Amsterdam Original 3i Ladies' - 2017,1,3,2017,659.99,1,Electra
4,258,Electra Amsterdam Royal 8i - 2017/2018,1,3,2018,1259.90,1,Electra


In [11]:
# We can also query tables selecting specific fields without Joining

no_join = pd.read_sql('select production.products.product_name, '
                      'production.products.model_year, '
                      'production.products.list_price, '
                      'production.brands.brand_name '
                      'from production.products, production.brands '
                      'where production.products.brand_id = production.brands.brand_id ', conn)
no_join.head()

,product_name,model_year,list_price,brand_name
0,Trek 820 - 2016,2016,379.99,Trek
1,Ritchey Timberwolf Frameset - 2016,2016,749.99,Ritchey
2,Surly Wednesday Frameset - 2016,2016,999.99,Surly
3,Trek Fuel EX 8 29 - 2016,2016,2899.99,Trek
4,Heller Shagamaw Frame - 2016,2016,1320.99,Heller


In [12]:
# Check to see if number of records is correct 

no_join.shape

(321, 4)

In [13]:
# We create virtual tables or subtables to avoid re-writing queries
# View subtable query

view = pd.read_sql('select * from vtabl ', conn)
view.head()

,product_name,model_year,brand_id,list_price
0,Trek 820 - 2016,2016,9,379.99
1,Ritchey Timberwolf Frameset - 2016,2016,5,749.99
2,Surly Wednesday Frameset - 2016,2016,8,999.99
3,Trek Fuel EX 8 29 - 2016,2016,9,2899.99
4,Heller Shagamaw Frame - 2016,2016,3,1320.99


#### Create Stored Procedure to avoid typing and copying and pasting queries:

    CREATE PROCEDURE MUFG_Sample
    AS
    Select *
    FROM production.products as t1
    LEFT JOIN production.brands as t2
    ON t1.brand_id = t2.brand_id
    ORDER BY product_name

In [14]:
# Query using stored procedure

sp = pd.read_sql('exec MUFG_Sample ', conn)
sp.head()

,product_id,product_name,brand_id,category_id,model_year,list_price,brand_id,brand_name
0,257,Electra Amsterdam Fashion 3i Ladies' - 2017/2018,1,3,2018,899.99,1,Electra
1,81,Electra Amsterdam Fashion 7i Ladies' - 2017,1,3,2017,1099.99,1,Electra
2,70,Electra Amsterdam Original 3i - 2015/2017,1,3,2017,659.99,1,Electra
3,82,Electra Amsterdam Original 3i Ladies' - 2017,1,3,2017,659.99,1,Electra
4,258,Electra Amsterdam Royal 8i - 2017/2018,1,3,2018,1259.90,1,Electra


In [15]:
# View dataset size

sp.shape

(321, 8)

In [16]:
# Since the query duplicated the "brand_id" column, let's remove it

new_sp= sp.loc[:,~sp.columns.duplicated()]
new_sp.head()

,product_id,product_name,brand_id,category_id,model_year,list_price,brand_name
0,257,Electra Amsterdam Fashion 3i Ladies' - 2017/2018,1,3,2018,899.99,Electra
1,81,Electra Amsterdam Fashion 7i Ladies' - 2017,1,3,2017,1099.99,Electra
2,70,Electra Amsterdam Original 3i - 2015/2017,1,3,2017,659.99,Electra
3,82,Electra Amsterdam Original 3i Ladies' - 2017,1,3,2017,659.99,Electra
4,258,Electra Amsterdam Royal 8i - 2017/2018,1,3,2018,1259.90,Electra


In [17]:
# Review dataset size

new_sp.shape

(321, 7)

#### Create View to avoid overusing system resources:

    CREATE VIEW vtabl
    AS
    Select *
    FROM production.products as t1
    LEFT JOIN production.brands as t2
    ON t1.brand_id = t2.brand_id
    ORDER BY product_name

In [21]:
# Check virtual table view

view_tbl = pd.read_sql('select * from vtabl ', conn)
view_tbl.head()

,product_name,model_year,brand_id,list_price
0,Trek 820 - 2016,2016,9,379.99
1,Ritchey Timberwolf Frameset - 2016,2016,5,749.99
2,Surly Wednesday Frameset - 2016,2016,8,999.99
3,Trek Fuel EX 8 29 - 2016,2016,9,2899.99
4,Heller Shagamaw Frame - 2016,2016,3,1320.99
